# Prompt Engineering with GitHub Models

This notebook demonstrates prompt engineering concepts using GitHub's Azure AI Inference service.

## Prerequisites

- Set `GITHUB_TOKEN` environment variable with a GitHub Personal Access Token
- Install required packages: `pip install azure-ai-inference`

## Setup

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

# Get GitHub token from environment
github_token = os.environ.get("GITHUB_TOKEN")
if not github_token:
    raise ValueError("GITHUB_TOKEN environment variable must be set")

# Initialize client
endpoint = "https://models.github.ai/inference"
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(github_token)
)

# Choose a model (GPT-4o, GPT-4o-mini, etc.)
model = "gpt-4o-mini"

print(f"✅ Connected to {endpoint}")
print(f"📦 Using model: {model}")

## 1. Vague vs. Specific Prompts

Demonstrating the importance of specificity in prompts.

In [ ]:
# Vague prompt
vague_response = client.complete(
    messages=[
        UserMessage(content="Write a function to validate email.")
    ],
    model=model,
    temperature=0.7
)

print("❌ VAGUE PROMPT RESULT:")
print("="*60)
print(vague_response.choices[0].message.content)
print("\n" + "="*60)

In [ ]:
# Specific prompt with context
specific_response = client.complete(
    messages=[
        UserMessage(content="""Write a Python function to validate email addresses with the following requirements:

Function name: validate_email
Parameters: email (str)
Returns: tuple (bool, str) - (is_valid, error_message)

Requirements:
- Check for @ symbol presence
- Validate domain has at least one dot
- No spaces allowed
- Local part (before @) must be 1-64 characters
- Domain must be 1-255 characters
- Return specific error messages for each validation failure

Example usage:
valid, msg = validate_email("user@example.com")  # (True, "")
valid, msg = validate_email("invalid")  # (False, "Missing @ symbol")
""")
    ],
    model=model,
    temperature=0.7
)

print("✅ SPECIFIC PROMPT RESULT:")
print("="*60)
print(specific_response.choices[0].message.content)
print("\n" + "="*60)

## 2. Using System Messages for Context

System messages set the behavior and context for the AI.

In [ ]:
# With system context
response = client.complete(
    messages=[
        SystemMessage(content="""You are an expert Python developer who writes secure, 
production-ready code following PEP 8 guidelines. Always include:
- Type hints
- Docstrings
- Error handling
- Input validation
"""),
        UserMessage(content="Write a function to parse a JSON file and extract user data.")
    ],
    model=model,
    temperature=0.7
)

print("🎯 RESULT WITH SYSTEM CONTEXT:")
print("="*60)
print(response.choices[0].message.content)
print("\n" + "="*60)

## 3. Few-Shot Learning with Examples

Providing examples to establish patterns.

In [ ]:
from azure.ai.inference.models import AssistantMessage

# Few-shot prompting
response = client.complete(
    messages=[
        SystemMessage(content="You convert natural language to SQL queries."),
        UserMessage(content="Show all users"),
        AssistantMessage(content="SELECT * FROM users;"),
        UserMessage(content="Find users named John"),
        AssistantMessage(content="SELECT * FROM users WHERE name = 'John';"),
        UserMessage(content="Count active users from California")
    ],
    model=model,
    temperature=0.3
)

print("📚 FEW-SHOT LEARNING RESULT:")
print("="*60)
print(response.choices[0].message.content)
print("\n" + "="*60)

## 4. Role-Based Prompting

Using specific roles to guide the AI's perspective.

In [ ]:
code_to_review = """
def login(username, password):
    user = db.execute(f"SELECT * FROM users WHERE username='{username}'")
    if user and user['password'] == password:
        session['user_id'] = user['id']
        return True
    return False
"""

# Security expert role
security_review = client.complete(
    messages=[
        SystemMessage(content="""You are a security expert reviewing code for vulnerabilities.
Focus on: SQL injection, authentication flaws, password handling, and session management.
Provide specific CVE references when applicable."""),
        UserMessage(content=f"Review this login function for security issues:\n\n{code_to_review}")
    ],
    model=model,
    temperature=0.5
)

print("🔒 SECURITY EXPERT REVIEW:")
print("="*60)
print(security_review.choices[0].message.content)
print("\n" + "="*60)

## 5. Contextual Chaining

Building context progressively across multiple prompts.

In [ ]:
conversation = []

# Step 1: Create base structure
conversation.append(SystemMessage(content="You are a Python expert building a REST API."))
conversation.append(UserMessage(content="Create a User model with id, email, and name fields using dataclasses."))

response1 = client.complete(messages=conversation, model=model, temperature=0.7)
conversation.append(AssistantMessage(content=response1.choices[0].message.content))

print("🔨 STEP 1: Base Model")
print("="*60)
print(response1.choices[0].message.content)
print("\n")

# Step 2: Add validation
conversation.append(UserMessage(content="Add validation methods to ensure email is valid and name is not empty."))
response2 = client.complete(messages=conversation, model=model, temperature=0.7)
conversation.append(AssistantMessage(content=response2.choices[0].message.content))

print("✅ STEP 2: With Validation")
print("="*60)
print(response2.choices[0].message.content)
print("\n")

# Step 3: Add repository
conversation.append(UserMessage(content="Create a UserRepository class with CRUD operations using this User model."))
response3 = client.complete(messages=conversation, model=model, temperature=0.7)

print("🗄️ STEP 3: Repository Pattern")
print("="*60)
print(response3.choices[0].message.content)
print("\n" + "="*60)

## 6. Constraining Outputs

Using constraints to control the format and content of responses.

In [ ]:
response = client.complete(
    messages=[
        SystemMessage(content="""You generate Python code with these STRICT constraints:
- Use type hints for all functions
- Maximum function length: 20 lines
- Include docstrings in Google style
- Use descriptive variable names (no single letters except loop indices)
- Prefer composition over inheritance
- Always handle exceptions explicitly"""),
        UserMessage(content="Create a function to calculate the Fibonacci sequence up to n terms.")
    ],
    model=model,
    temperature=0.7
)

print("⚖️ CONSTRAINED OUTPUT:")
print("="*60)
print(response.choices[0].message.content)
print("\n" + "="*60)

## 7. Token Usage Tracking

Monitor token consumption to optimize costs.

In [ ]:
# Compare token usage between vague and specific prompts
def get_token_usage(messages, model_name):
    response = client.complete(messages=messages, model=model_name, temperature=0.7)
    usage = response.usage
    return {
        'prompt_tokens': usage.prompt_tokens,
        'completion_tokens': usage.completion_tokens,
        'total_tokens': usage.total_tokens,
        'response': response.choices[0].message.content
    }

# Vague prompt
vague_result = get_token_usage(
    [UserMessage(content="Write a sorting function")],
    model
)

# Specific prompt
specific_result = get_token_usage(
    [UserMessage(content="""Write a Python function to sort a list of dictionaries by a specified key.
    
Function: sort_dict_list(items: List[dict], key: str, reverse: bool = False) -> List[dict]
- Handle missing keys gracefully (move to end)
- Support nested keys using dot notation (e.g., 'user.age')
- Include type hints and docstring
""")],
    model
)

print("📊 TOKEN USAGE COMPARISON:")
print("="*60)
print(f"Vague Prompt:")
print(f"  Input:  {vague_result['prompt_tokens']} tokens")
print(f"  Output: {vague_result['completion_tokens']} tokens")
print(f"  Total:  {vague_result['total_tokens']} tokens")
print()
print(f"Specific Prompt:")
print(f"  Input:  {specific_result['prompt_tokens']} tokens")
print(f"  Output: {specific_result['completion_tokens']} tokens")
print(f"  Total:  {specific_result['total_tokens']} tokens")
print()
print(f"Difference: {specific_result['total_tokens'] - vague_result['total_tokens']} tokens")
print(f"Quality improvement: Specific prompt provides detailed requirements")
print("="*60)

## Summary

Key takeaways from prompt engineering:

1. **Be Specific**: Detailed prompts yield better, more predictable results
2. **Use System Messages**: Set context and constraints upfront
3. **Provide Examples**: Few-shot learning establishes patterns
4. **Use Roles**: Guide the AI's perspective and expertise
5. **Chain Context**: Build complexity progressively
6. **Set Constraints**: Control output format and style
7. **Monitor Tokens**: Track usage for cost optimization

## Next Steps

- Explore `04-context-optimization.ipynb` for token optimization strategies
- Learn about RAG patterns in `05-rag-and-retrieval.ipynb`
- Build complete pipelines in `06-context-pipeline.ipynb`